In [10]:
# pip install runwayml requests pillow
from runwayml import RunwayML, TaskFailedError
import base64, mimetypes, io, requests
from PIL import Image

API_KEY = "key_c6d1952d237e03037fff2dce161f83f8d654ba48fc6032d3798157cb7654090e64f7b5d935349079c2ec04036c6f30c8a6afe92db66eb3756ef8cdb023a08582"

IMAGE_PATH = r"C:\Users\76135\Desktop\ADSYS-Cardboard-AI-Detection\Versions\V2 Nanobanana Sythetic cardboard bundle dataset\Nano_banana\dataset\standard images  (8).png"
OUTPUT_PATH = "cardboard_right_to_left.mp4"

# Minimal, stable, single-action prompt
PROMPT = (
    "The objects tavels slowly over a conveyor."
)

def image_to_data_uri(path, max_bytes=5*1024*1024 - 2048):
    # Prefer original; if too big for data URI, re-encode as JPEG until it fits (~5 MB base64 cap)
    mime = mimetypes.guess_type(path)[0] or "image/png"
    with open(path, "rb") as f:
        raw = f.read()
    uri = f"data:{mime};base64,{base64.b64encode(raw).decode('utf-8')}"
    if len(uri.encode("utf-8")) < max_bytes:
        return uri

    img = Image.open(path).convert("RGB")
    for q in (95, 90, 85, 80, 75, 70, 60):
        buf = io.BytesIO()
        img.save(buf, format="JPEG", quality=q, optimize=True)
        uri = f"data:image/jpeg;base64,{base64.b64encode(buf.getvalue()).decode('utf-8')}"
        if len(uri.encode("utf-8")) < max_bytes:
            return uri
    raise ValueError("Image too large for data URI; consider hosting it and passing the https URL.")

def main():
    client = RunwayML(api_key=API_KEY)
    data_uri = image_to_data_uri(IMAGE_PATH)

    try:
        task = client.image_to_video.create(
            model="gen4_turbo",
            prompt_image=data_uri,
            prompt_text=PROMPT,
            ratio="960:960",
            duration=5,
            # seed=42,  # helps with consistency across retries
            extra_headers={"X-Runway-Version": "2024-11-06"},
        ).wait_for_task_output()

        video_url = task.output[0]
        r = requests.get(video_url, timeout=300)
        r.raise_for_status()
        with open(OUTPUT_PATH, "wb") as f:
            f.write(r.content)
        print(f"Saved: {OUTPUT_PATH}")
    except TaskFailedError as e:
        print("Generation failed:")
        print(e.task_details)

if __name__ == "__main__":
    main()


Saved: cardboard_right_to_left.mp4
